In [2]:
# Importo linrerias necesarias
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import det_curve
from sklearn.metrics import accuracy_score

In [3]:
# Preparo los datos
data = pd.read_csv('C:/Users/Universidad/Proyecto_ASMI/archivo_modificado_final.csv')

# Definir el umbral de no nulos
umbral = 0.90  

# Calcular el porcentaje de valores no nulos por columna
porcentaje_no_nulos = data.notnull().mean()

# Filtrar las columnas que cumplen con el umbral
columnas_filtradas = porcentaje_no_nulos[porcentaje_no_nulos > umbral].index

# Crear un nuevo DataFrame con solo esas columnas
df_filtrado = data[columnas_filtradas]

# Mostrar el resultado
print(df_filtrado.shape)

(7440, 63)


In [4]:
# Definir el umbral de no nulos
umbral_filas = 0.30  

# Calcular el número mínimo de valores no nulos requeridos por fila
min_no_nulos = int(umbral_filas * df_filtrado.shape[1])

# Eliminar las filas que no cumplen con el umbral
df_filtrado_filas = df_filtrado.dropna(thresh=min_no_nulos)

# Mostrar el resultado
print(df_filtrado_filas.shape)

(7440, 63)


In [5]:
# Saco la mediana de cada columna y la relleno en los valores nulos
df_filtrado_filas2 = df_filtrado_filas.drop(columns=['usuario'])
df_filtrado_filas2 = df_filtrado_filas2.fillna(df_filtrado_filas2.mean())
df_filtrado_filas2 = df_filtrado_filas2.round(2)

# Reemplazar los valores inf por nan
df_filtrado_filas2 = df_filtrado_filas2.replace([np.inf, -np.inf], np.nan)
# Reeemplazar los valores nan por la media de la columna
df_filtrado_filas2 = df_filtrado_filas2.fillna(df_filtrado_filas2.mean())

In [6]:
# Separar características y etiquetas
X = df_filtrado_filas2.drop(columns=['usuario_num', 'muestra', 'sesion', 'digito']).values  # Todas las columnas excepto la tercera como características
y = df_filtrado_filas2.iloc[:, 1].values  # Segunda columna como etiqueta

# Normalizar características
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Codificar etiquetas
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Primero, extraemos los identificadores únicos de los usuarios
usuarios_unicos = df_filtrado_filas2['usuario_num'].unique()

# Seleccionar aleatoriamente 20 usuarios para el entrenamiento
np.random.seed(42)  # Para garantizar la reproducibilidad
usuarios_entrenamiento = np.random.choice(usuarios_unicos, 20, replace=False)

# Filtrar los datos para entrenamiento (solo los usuarios seleccionados)
X_train = X[df_filtrado_filas2['usuario_num'].isin(usuarios_entrenamiento)]
y_train = y[df_filtrado_filas2['usuario_num'].isin(usuarios_entrenamiento)]

# Filtrar los datos para test (el resto de los usuarios)
X_test = X[~df_filtrado_filas2['usuario_num'].isin(usuarios_entrenamiento)]
y_test = y[~df_filtrado_filas2['usuario_num'].isin(usuarios_entrenamiento)]

# Construir el modelo ANN
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Precisión en test: {test_acc * 100:.2f}%')

# Obtener predicciones de probabilidad
y_probs = model.predict(X_test)

# Obtener las predicciones del modelo
y_pred = np.argmax(model.predict(X_test), axis=1)

# Calcular la precisión de cada dígito
for i in range(10):
    # Filtrar los valores para el dígito i
    y_true_digit = (y_test == i)
    y_pred_digit = (y_pred == i)

    # Calcular precisión para ese dígito
    digit_accuracy = accuracy_score(y_true_digit, y_pred_digit)
    print(f'Precisión para el dígito {i}: {digit_accuracy * 100:.2f}%')


Epoch 1/30
50/50 [==============================] - 1s 10ms/step - loss: 1.8983 - accuracy: 0.3738 - val_loss: 1.4848 - val_accuracy: 0.5889
Epoch 2/30
50/50 [==============================] - 0s 5ms/step - loss: 1.0805 - accuracy: 0.6938 - val_loss: 0.9605 - val_accuracy: 0.7435
Epoch 3/30
50/50 [==============================] - 1s 13ms/step - loss: 0.6670 - accuracy: 0.7962 - val_loss: 0.7268 - val_accuracy: 0.7964
Epoch 4/30
50/50 [==============================] - 1s 12ms/step - loss: 0.4886 - accuracy: 0.8519 - val_loss: 0.6565 - val_accuracy: 0.8099
Epoch 5/30
50/50 [==============================] - 1s 11ms/step - loss: 0.3974 - accuracy: 0.8750 - val_loss: 0.6168 - val_accuracy: 0.8192
Epoch 6/30
50/50 [==============================] - 1s 12ms/step - loss: 0.3272 - accuracy: 0.8931 - val_loss: 0.5709 - val_accuracy: 0.8351
Epoch 7/30
50/50 [==============================] - 1s 12ms/step - loss: 0.2702 - accuracy: 0.9225 - val_loss: 0.5546 - val_accuracy: 0.8409
Epoch 8/30
50/